# ROIC Analysis ver4


In [2]:
%load_ext autoreload
%autoreload 2

import sqlite3
import warnings
from pathlib import Path

import factset_downloaded_data_utils as f_db_utils
import numpy as np
import pandas as pd
import ROIC_make_data_files_ver2 as roic_utils
from database_utils import get_table_names

warnings.simplefilter("ignore")

UNIVERSE_CODE = "MSXJPN_AD"

ROOT_DIR = Path().cwd().parent
DATA_DIR = ROOT_DIR / "data"
FACTSET_DIR = DATA_DIR / "Factset"
BPM_DIR = DATA_DIR / "BPM"
INDEX_DIR = FACTSET_DIR / f"Financials/{UNIVERSE_CODE}"
INDEX_CONSTITUENTS_DIR = FACTSET_DIR / "Index_Constituents"


db_path = INDEX_DIR / "Financials_and_Price.db"
factset_index_db_path = INDEX_CONSTITUENTS_DIR / "Index_Constituents.db"
bpm_db_path = BPM_DIR / "Index_Constituents.db"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## BPM と Factset からダウンロードしたデータを sqlite3 に保存

-   インデックス別にテーブルを作成する
-   元データは"Index_Constituents_with_Factset_code-compressed-\*.paruqet" -> 圧縮して送信した
-   BPM から取得した構成比や銘柄 ID などのデータと、Factset でダウンロードした seol, cusip, isin, code_jp にそれぞれ対応する P_SYMBOL および FG_COMPANY_NAME を格納したデータ。


In [ ]:
compressed_files = list(
    INDEX_CONSTITUENTS_DIR.glob(
        "Index_Constituents_with_Factset_code-compressed-*.parquet"
    )
)

dfs = [pd.read_parquet(f) for f in compressed_files]
df = (
    pd.concat(dfs)
    .assign(
        date=lambda x: pd.to_datetime(x["date"]),
        SEDOL=lambda x: x["SEDOL"].astype(str),
    )
    .replace("N/A", np.nan)
)
df[["Holdings", "Weight (%)", "Mkt Value"]] = df[
    ["Holdings", "Weight (%)", "Mkt Value"]
].astype(float)

head_cols = ["Universe", "Universe_code_BPM", "date"]
other_cols = [col for col in df.columns if col not in head_cols]
df = df.reindex(columns=head_cols + other_cols).sort_values(
    ["Universe", "date", "Name"], ignore_index=True
)

for universe_code in df["Universe_code_BPM"].unique():
    df_slice = df.loc[df["Universe_code_BPM"] == universe_code].reset_index(drop=True)
    f_db_utils.store_to_database(
        df=df_slice,
        db_path=factset_index_db_path,
        table_name=universe_code,
        unique_cols=["date", "Name", "Asset ID"],
    )

table_names = get_table_names(db_path=factset_index_db_path)
display(table_names)

テーブル 'MSSUD' は存在しません。新しいテーブルとして、すべての 47072 行を追加します。
データの書き込みが完了しました。
テーブル 'MSASD' は存在しません。新しいテーブルとして、すべての 319851 行を追加します。
データの書き込みが完了しました。
テーブル 'MSACAPFAD' は存在しません。新しいテーブルとして、すべての 344002 行を追加します。
データの書き込みが完了しました。
テーブル 'MSAPFXJ_AD' は存在しません。新しいテーブルとして、すべての 247340 行を追加します。
データの書き込みが完了しました。
テーブル 'MSCAFXJAD' は存在しません。新しいテーブルとして、すべての 223596 行を追加します。
データの書き込みが完了しました。
テーブル 'MSAWIF_AD' は存在しません。新しいテーブルとして、すべての 788150 行を追加します。
データの書き込みが完了しました。
テーブル 'MSCIHKGD' は存在しません。新しいテーブルとして、すべての 11887 行を追加します。
データの書き込みが完了しました。
テーブル 'MSCIINDD' は存在しません。新しいテーブルとして、すべての 23987 行を追加します。
データの書き込みが完了しました。
テーブル 'MSFIDND' は存在しません。新しいテーブルとして、すべての 7701 行を追加します。
データの書き込みが完了しました。
テーブル 'MSXJPN_AD' は存在しません。新しいテーブルとして、すべての 399723 行を追加します。
データの書き込みが完了しました。
テーブル 'MSFKORD' は存在しません。新しいテーブルとして、すべての 29542 行を追加します。
データの書き込みが完了しました。
テーブル 'MSFMALD' は存在しません。新しいテーブルとして、すべての 15061 行を追加します。
データの書き込みが完了しました。
テーブル 'MSFPHID' は存在しません。新しいテーブルとして、すべての 5510 行を追加します。
データの書き込みが完了しました。
テーブル 'MSCISIND' は存在しません。新しいテーブルとして、すべての 8856 行を追加します。
データの書き込みが完

['MSSUD',
 'MSASD',
 'MSACAPFAD',
 'MSAPFXJ_AD',
 'MSCAFXJAD',
 'MSAWIF_AD',
 'MSCIHKGD',
 'MSCIINDD',
 'MSFIDND',
 'MSXJPN_AD',
 'MSFKORD',
 'MSFMALD',
 'MSFPHID',
 'MSCISIND',
 'MSFTAID',
 'MSFTHAD']

In [ ]:
conn = sqlite3.connect(factset_index_db_path)
df = pd.read_sql("SELECT * FROM MSXJPN_AD", con=conn)
df["P_SYMBOL_missing"] = df["P_SYMBOL"].isna()
display(df)

g = df.groupby(["date", "P_SYMBOL_missing"])["Weight (%)"].agg(["count", "sum"])
display(g)

,Universe,Universe_code_BPM,date,Name,Bloomberg Ticker,BloombergID,Asset ID,Asset ID Type,SEDOL,Country,...,P_SYMBOL_CUSIP,ISIN,FG_COMPANY_NAME_ISIN,P_SYMBOL_ISIN,CODE_JP,FG_COMPANY_NAME_CODE_JP,P_SYMBOL_CODE_JP,P_SYMBOL,FG_COMPANY_NAME,P_SYMBOL_missing
0,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31 00:00:00,21ST CENTURY FOX,None,None,AUSBIN2,BARRAID,662075,AUS,...,None,None,None,None,None,None,None,FOXLV-AU,Twenty-First Century Fox Inc. Class A CDI,False
1,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31 00:00:00,21ST CENTURY FOX,None,None,AUSBIN1,BARRAID,688692,AUS,...,None,None,None,None,None,None,None,FOX-AU,Twenty-First Century Fox Inc. Class B CDI,False
2,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31 00:00:00,3I GROUP PLC,None,None,UKIENL1,BARRAID,0888693,GBR,...,TGOPF-US,GB0008886938,スリーアイ・グループ,III-GB,None,None,None,III-GB,スリーアイ・グループ,False
3,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31 00:00:00,3M CO,None,None,USAJ8P1,BARRAID,2595708,USA,...,MMM-US,US6040591058,3Mカンパニー,MMM-US,None,None,None,MMM-US,3Mカンパニー,False
4,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31 00:00:00,ABB LTD,None,None,SWIAAN1,BARRAID,5661190,CHE,...,None,CH0003846620,ABB,ABBN-CH,None,None,None,ABBN-CH,ABB,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399718,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-09-30 00:00:00,ZIMMER BIOMET HOLDINGS INC,None,None,USA4JT1,BARRAID,2783815,USA,...,ZBH-US,US98956P1021,ジンマー・バイオメット・ホールディングス,ZBH-US,None,None,None,ZBH-US,ジンマー・バイオメット・ホールディングス,False
399719,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-09-30 00:00:00,ZOETIS INC,None,None,USBANZ1,BARRAID,B95WG16,USA,...,ZTS-US,US98978V1035,ゾエティス Class A,ZTS-US,None,None,None,ZTS-US,ゾエティス Class A,False
399720,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-09-30 00:00:00,ZOOM COMMUNICATIONS INC,None,None,USBEOV1,BARRAID,BGSP7M9,USA,...,ZM-US,US98980L1017,ズーム・ビデオ・コミュニケーションズ Class A,ZM-US,None,None,None,ZM-US,ズーム・ビデオ・コミュニケーションズ Class A,False
399721,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-09-30 00:00:00,ZSCALER INC,None,None,USBDYI1,BARRAID,BZ00V34,USA,...,ZS-US,US98980G1022,ゼットスケイラー,ZS-US,None,None,None,ZS-US,ゼットスケイラー,False


count         sum
date                P_SYMBOL_missing                   
2000-01-31 00:00:00 False               991   97.959673
                    True                 74    2.040327
2000-02-29 00:00:00 False               989   97.900800
                    True                 73    2.099194
2000-03-31 00:00:00 False               989   98.129503
...                                     ...         ...
2025-05-31 00:00:00 True                  1    0.000228
2025-06-30 00:00:00 False              1142   99.999989
2025-07-31 00:00:00 False              1139  100.000012
2025-08-31 00:00:00 False              1140  100.000009
2025-09-30 00:00:00 False              1140   99.999991

[400 rows x 2 columns]

## Factset からダウンロードしたデータをまとめる

1. まず Financials および Price のデータをデータベースに格納


In [ ]:
file_list = list(INDEX_DIR.glob("Financials_and_Price-compressed-*.parquet"))
dfs = [pd.read_parquet(f) for f in file_list]
df = (
    pd.concat(dfs)
    .sort_values(["variable", "P_SYMBOL", "date"], ignore_index=True)
    .assign(value=lambda x: x["value"].astype(float))
)

for variable in df["variable"].unique():
    df_slice = df.loc[df["variable"] == variable]
    f_db_utils.store_to_database(
        df=df_slice,
        db_path=db_path,
        table_name=variable,
        unique_cols=["date", "P_SYMBOL"],
    )

table_names = get_table_names(db_path=db_path)
display(table_names)

テーブル 'FF_ASSETS' は存在しません。新しいテーブルとして、すべての 515864 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'FF_BPS' は存在しません。新しいテーブルとして、すべての 515864 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'FF_BPS_TANG' は存在しません。新しいテーブルとして、すべての 515864 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'FF_CAPEX' は存在しません。新しいテーブルとして、すべての 515864 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'FF_CASH_ST' は存在しません。新しいテーブルとして、すべての 515864 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'FF_COGS' は存在しません。新しいテーブルとして、すべての 515864 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'FF_COM_EQ' は存在しません。新しいテーブルとして、すべての 515864 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'FF_CURR_RATIO' は存在しません。新しいテーブルとして、すべての 515864 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'FF_DEBT' は存在しません。新しいテーブルとして、すべての 515864 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'FF_DEBT_ENTRPR_VAL' は存在しません。新しいテーブルとして、すべての 515864 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'FF_DEBT_EQ' は存在しません。新しいテーブルとして、すべての 515864 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'FF_DEBT_LT' は存在しません。新しいテーブルとして、すべての 515864 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'FF_DEBT_ST' は存在しません。新しいテーブルとして、すべての 

['Financials',
 'Price',
 'FF_ASSETS',
 'FF_BPS',
 'FF_BPS_TANG',
 'FF_CAPEX',
 'FF_CASH_ST',
 'FF_COGS',
 'FF_COM_EQ',
 'FF_CURR_RATIO',
 'FF_DEBT',
 'FF_DEBT_ENTRPR_VAL',
 'FF_DEBT_EQ',
 'FF_DEBT_LT',
 'FF_DEBT_ST',
 'FF_DEP_AMORT_EXP',
 'FF_DIV_YLD',
 'FF_DPS',
 'FF_EBITDA_OPER',
 'FF_EBITDA_OPER_MGN',
 'FF_EBIT_OPER',
 'FF_EBIT_OPER_MGN',
 'FF_ENTRPR_VAL_EBITDA_OPER',
 'FF_ENTRPR_VAL_EBIT_OPER',
 'FF_ENTRPR_VAL_SALES',
 'FF_EPS',
 'FF_EPS_DIL',
 'FF_FREE_CF',
 'FF_FREE_PS_CF',
 'FF_GROSS_INC',
 'FF_GROSS_MGN',
 'FF_INC_TAX',
 'FF_INT_EXP_NET',
 'FF_LIABS',
 'FF_LIABS_SHLDRS_EQ',
 'FF_MIN_INT_ACCUM',
 'FF_NET_DEBT',
 'FF_NET_INC',
 'FF_NET_MGN',
 'FF_OPER_CF',
 'FF_OPER_INC',
 'FF_OPER_MGN',
 'FF_OPER_PS_NET_CF',
 'FF_PAY_OUT_RATIO',
 'FF_PBK',
 'FF_PE',
 'FF_PFD_STK',
 'FF_PPE_NET',
 'FF_PSALES',
 'FF_PTX_INC',
 'FF_PTX_MGN',
 'FF_QUICK_RATIO',
 'FF_ROA',
 'FF_ROE',
 'FF_ROIC',
 'FF_ROTC',
 'FF_SALES',
 'FF_SALES_PS',
 'FF_SGA',
 'FF_SHLDRS_EQ',
 'FF_STK_OPT_EXP',
 'FF_STK_PURCH_CF

### リターンのテーブルを作成


In [ ]:
conn = sqlite3.connect(db_path)
df = pd.read_sql(
    "SELECT date, P_SYMBOL, value FROM FG_PRICE",
    con=conn,
    parse_dates=["date"],  # read_sql側で日付型に変換
    index_col="date",  # 同時にインデックスに設定
)
# value の型変換とリネーム
df["FG_PRICE"] = df["value"].astype(float)
df = df.drop(columns=["value"]).sort_index()

# 1. 元のdfをP_SYMBOLとdateを列に戻す
df_reset = df.reset_index()

# 2. 全銘柄リストと全日付リストを取得
all_symbols = df_reset["P_SYMBOL"].unique()
all_dates = df_reset["date"].unique()

# 3. 全銘柄×全日付の組み合わせ(MultiIndex)を作成
new_index = pd.MultiIndex.from_product(
    [all_symbols, all_dates], names=["P_SYMBOL", "date"]
)

# 4. 元のデータをMultiIndexにセットし直し、存在しない組み合わせをNaNで埋める
#    これが「整然化」されたデータ
df_regular = df_reset.set_index(["P_SYMBOL", "date"]).reindex(new_index)

# 5. この整然化されたデータに対してリターンを計算する
#    dateでソートする必要がある
df_regular = df_regular.sort_index(level="date")

for period_month in [1, 3, 6, 12, 36, 60]:
    period_name = (
        f"{int(period_month // 12)}Y" if period_month >= 36 else f"{period_month}M"
    )

    # 通常のリターンとforwardリターンを計算
    df_regular[f"Return_{period_name}"] = df_regular.groupby("P_SYMBOL")[
        "FG_PRICE"
    ].pct_change(period_month)
    df_regular[f"Forward_Return_{period_name}"] = df_regular.groupby("P_SYMBOL")[
        f"Return_{period_name}"
    ].shift(-period_month)

    # 年率化したカラムを追加
    if period_month > 12:
        df_regular[f"Return_{period_name}_annlzd"] = df_regular[
            f"Return_{period_name}"
        ].div(int(period_month // 12))
        df_regular[f"Forward_Return_{period_name}_annlzd"] = df_regular[
            f"Forward_Return_{period_name}"
        ].div(int(period_month // 12))
    else:
        df_regular[f"Return_{period_name}_annlzd"] = df_regular[
            f"Return_{period_name}"
        ].div(int(12 // period_month))
        df_regular[f"Forward_Return_{period_name}_annlzd"] = df_regular[
            f"Forward_Return_{period_name}"
        ].div(int(12 // period_month))


# --------------
# データチェック
# 銘柄によってはdateが1カ月ずつ連続でデータがあるとは限らない
# FG_PRICEがない場合にpct_changeを素直に実行するとリターンの期間が他の銘柄とずれる
# そのため、全dateの長さと銘柄ごとのdateの長さを比較する
# --------------

df_check = df_regular.reset_index()
symbol_date_counts = df_check.groupby("P_SYMBOL")["date"].nunique()
all_date_len = len(df_check["date"].unique())
not_enough_len_symbols = symbol_date_counts[symbol_date_counts != all_date_len].index
if len(not_enough_len_symbols) > 0:
    display(not_enough_len_symbols)
else:  # 問題なければデータベースに保存
    print("問題なし")
    df_regular.reset_index(inplace=True)
    display(df_regular.head(5))
    for col in [
        s
        for s in df_regular.columns
        if s.startswith("Return") or s.startswith("Forward_Return")
    ]:
        df_slice = (
            df_regular[["date", "P_SYMBOL", col]]
            .rename(columns={col: "value"})
            .assign(variable=col)
        )
        df_slice["value"] = df_slice["value"].astype(float)
        df_slice["date"] = pd.to_datetime(df_slice["date"])
        f_db_utils.store_to_database(df=df_slice, db_path=db_path, table_name=col)

問題なし


,P_SYMBOL,date,FG_PRICE,Return_1M,Forward_Return_1M,Return_1M_annlzd,Forward_Return_1M_annlzd,Return_3M,Forward_Return_3M,Return_3M_annlzd,...,Return_12M_annlzd,Forward_Return_12M_annlzd,Return_3Y,Forward_Return_3Y,Return_3Y_annlzd,Forward_Return_3Y_annlzd,Return_5Y,Forward_Return_5Y,Return_5Y_annlzd,Forward_Return_5Y_annlzd
0,0HSW-GB,2005-08-31,3.225569,NaN,0.127660,NaN,0.010638,NaN,0.342199,NaN,...,NaN,0.511361,NaN,0.847736,NaN,0.282579,NaN,0.847736,NaN,0.169547
1,0II3.XX1-GB,2005-08-31,4.166256,NaN,0.014796,NaN,0.001233,NaN,0.060419,NaN,...,NaN,0.093151,NaN,0.115485,NaN,0.038495,NaN,0.115485,NaN,0.023097
2,0MDJ-GB,2005-08-31,6.092351,NaN,0.045704,NaN,0.003809,NaN,0.015539,NaN,...,NaN,0.071298,NaN,0.416530,NaN,0.138843,NaN,0.416530,NaN,0.083306
3,0N1N-GB,2005-08-31,5.655000,NaN,0.038904,NaN,0.003242,NaN,0.015031,NaN,...,NaN,0.337754,NaN,0.366932,NaN,0.122311,NaN,0.366932,NaN,0.073386
4,0N3I-GB,2005-08-31,2.810000,NaN,0.028470,NaN,0.002372,NaN,0.025801,NaN,...,NaN,-0.150356,NaN,0.153025,NaN,0.051008,NaN,0.154448,NaN,0.030890


テーブル 'Return_1M' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Forward_Return_1M' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Return_1M_annlzd' は存在しません。新しいテーブルとして、すべての 516818 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'Forward_Return_1M_annlzd' は存在しません。新しいテーブルとして、すべての 516818 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'Return_3M' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Forward_Return_3M' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Return_3M_annlzd' は存在しません。新しいテーブルとして、すべての 516818 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'Forward_Return_3M_annlzd' は存在しません。新しいテーブルとして、すべての 516818 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'Return_6M' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Forward_Return_6M' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Return_6M_annlzd' は存在しません。新しいテーブルとして、すべての 516818 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'Forward_Return_6M_annlzd' は存在しません。新しいテーブルとして、すべての 516818 行を追加します。
  -> データの書き込みが完了しました。
テーブル 'Return_12M' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Forward_Return_12M' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Return_12M_annlzd' は存在しません。新しいテーブルとし

In [ ]:
## Make Return Date(Store to database)

db_path = DATA_DIR / "MSCI_KOKUSAI_Price_Daily.db"
conn = sqlite3.connect(db_path)
query = """
    SELECT
        *
    FROM
        FG_PRICE_Daily
    WHERE
        date IN(
                SELECT
                    MAX(date)
                FROM
                    FG_PRICE_Daily
                GROUP BY
                    strftime('%Y-%m', date)
            )
    ORDER BY
        date
"""
df = (
    pd.read_sql(query, con=conn, parse_dates=["date"])
    .drop(columns=["variable"])
    .rename(columns={"P_SYMBOL": "Symbol", "value": "FG_PRICE"})
)
conn.close()

# Symbolごとに、日付が最大となる行のインデックスを取得
# ->　最後の日付が中途半端(2025-8-25など)となっている場合があるため、そのような日付を削除する
idx_to_drop = df.groupby("Symbol")["date"].idxmax()
df = df.drop(idx_to_drop)
display(df)


df_return = df.copy()

if roic_utils.check_all_months_exist(df_return):
    df_return["Return_Ann_1M"] = df_return.groupby(["Symbol"])["FG_PRICE"].transform(
        lambda x: 12 * np.log(x / x.shift(1))
    )
    df_return["Return_Ann_3M"] = df_return.groupby(["Symbol"])["FG_PRICE"].transform(
        lambda x: 4 * np.log(x / x.shift(3))
    )
    df_return["Return_Ann_6M"] = df_return.groupby(["Symbol"])["FG_PRICE"].transform(
        lambda x: 2 * np.log(x / x.shift(6))
    )
    df_return["Return_Ann_1Y"] = df_return.groupby(["Symbol"])["FG_PRICE"].transform(
        lambda x: np.log(x / x.shift(12))
    )
    df_return["Return_Ann_3Y"] = df_return.groupby(["Symbol"])["FG_PRICE"].transform(
        lambda x: np.log(x / x.shift(36)) / 3
    )
    df_return["Return_Ann_5Y"] = df_return.groupby(["Symbol"])["FG_PRICE"].transform(
        lambda x: np.log(x / x.shift(60)) / 5
    )

    df_return = pd.melt(
        df_return.drop(columns=["FG_PRICE"]),
        id_vars=["date", "Symbol"],
        value_vars=[
            col for col in df_return.columns.tolist() if col.startswith("Return_")
        ],
        var_name="variable",
    )

    # export
    db_path = DATA_DIR / "MSCI_KOKUSAI_Return.db"
    conn = sqlite3.connect(db_path)
    df_return.to_sql("Annualized_Return", con=conn, index=False, if_exists="replace")
    # append_diff_to_sqlite(
    #     db_path=db_path, table_name="Annualized_Return", df_new=df_return
    # )
    conn.close()
    display(df_return)
    del df_return, df
else:
    raise

## データベース内容確認


In [ ]:
table_names = get_table_names(db_path=db_path)
display(table_names)

['Financials', 'Price']

In [ ]:
conn = sqlite3.connect(bpm_db_path)
df = pd.read_sql("SELECT * FROM MSXJPN_AD", con=conn)
display(df)

,Universe,Universe_code_BPM,date,Name,Bloomberg Ticker,BloombergID,Asset ID,Asset ID Type,CODE_JP,CUSIP,SEDOL,ISIN,Country,GICS Sector,GICS Industry,GICS Industry Group,GICS Sub-Industry,Holdings,Weight (%),Mkt Value
0,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31 00:00:00,21ST CENTURY FOX,None,None,AUSBIN1,BARRAID,None,None,688692,None,AUS,Consumer Discretionary,Media,Media,Movies & Entertainment,2.022644e+09,0.141024,2.547608e+12
1,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31 00:00:00,21ST CENTURY FOX,None,None,AUSBIN2,BARRAID,None,None,662075,None,AUS,Consumer Discretionary,Media,Media,Movies & Entertainment,2.134869e+09,0.130405,2.355792e+12
2,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31 00:00:00,3I GROUP PLC,None,None,UKIENL1,BARRAID,None,G4708P104,0888693,GB0008886938,GBR,Financials,Financial Services,Financial Services,Diversified Financial Services,5.954320e+08,0.058274,1.052726e+12
3,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31 00:00:00,3M CO,None,None,USAJ8P1,BARRAID,None,604059105,2595708,US6040591058,USA,Industrials,Industrial Conglomerates,Capital Goods,Industrial Conglomerates,4.013340e+08,0.222650,4.022205e+12
4,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31 00:00:00,ABB LTD,None,None,SWIAAN1,BARRAID,None,H7303M102,5661190,CH0003846620,CHE,Industrials,Electrical Equipment,Capital Goods,Heavy Electrical Equipment,3.000024e+08,0.197368,3.565481e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399718,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-09-30 00:00:00,ZIMMER BIOMET HOLDINGS INC,None,None,USA4JT1,BARRAID,None,98956P102,2783815,US98956P1021,USA,Health Care,Health Care Equipment & Supplies,Health Care Equipment & Services,Health Care Equipment,1.978479e+08,0.025576,2.878088e+12
399719,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-09-30 00:00:00,ZOETIS INC,None,None,USBANZ1,BARRAID,None,98978V103,B95WG16,US98978V1035,USA,Health Care,Pharmaceuticals,Pharmaceuticals Biotechnology & Life Sciences,Pharmaceuticals,4.452081e+08,0.085493,9.620621e+12
399720,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-09-30 00:00:00,ZOOM COMMUNICATIONS INC,None,None,USBEOV1,BARRAID,None,98980L101,BGSP7M9,US98980L1017,USA,Information Technology,Software,Software & Services,Application Software,2.472661e+08,0.026772,3.012693e+12
399721,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-09-30 00:00:00,ZSCALER INC,None,None,USBDYI1,BARRAID,None,98980G102,BZ00V34,US98980G1022,USA,Information Technology,Software,Software & Services,Systems Software,1.012022e+08,0.039800,4.478732e+12


## ROIC(ROE) + Security Code

-   セクター中立
-   金融セクターのみ ROIC の代わりに ROE を使用（ただしデータフレームのカラム名は ROIC で表記）


In [ ]:
# --- get ROIC and ROE data ---
conn = sqlite3.connect(db_path)
df_roic = (
    (pd.read_sql(sql="SELECT * FROM FF_ROIC", con=conn))
    .assign(date=lambda row: pd.to_datetime(row["date"]))
    .drop(columns=["variable"])
    .rename(columns={"P_SYMBOL": "Symbol", "value": "FF_ROIC"})
)
df_roe = (
    (pd.read_sql("SELECT * FROM FF_ROE", con=conn))
    .assign(date=lambda row: pd.to_datetime(row["date"]))
    .drop(columns=["variable"])
    .rename(columns={"P_SYMBOL": "Symbol", "value": "FF_ROE"})
)
df_roic = pd.merge(df_roic, df_roe, on=["date", "Symbol"], how="left").assign(
    date=lambda row: pd.to_datetime(row["date"]) + pd.tseries.offsets.MonthEnd(0)
)
del df_roe

# --- get security info data ---
conn = sqlite3.connect(factset_index_db_path)
query = f"""
    SELECT
        `date`, `P_SYMBOL`, `FG_COMPANY_NAME`, `GICS Sector`, `GICS Industry`, `Weight (%)`, `Mkt Value`
    FROM
        {UNIVERSE_CODE}
"""
security_info = (
    pd.read_sql(query, con=conn)
    .rename(columns={"P_SYMBOL": "Symbol"})
    .assign(
        date=lambda row: pd.to_datetime(row["date"]),
    )
)


# --- merge ---

df_roic_merged = (
    pd.merge(df_roic, security_info, on=["date", "Symbol"], how="left")
    .assign(
        ROIC=lambda x: np.where(
            x["GICS Sector"] == "Financials", x["FF_ROE"], x["FF_ROIC"]
        )
    )
    .dropna(subset=["Weight (%)", "ROIC"], how="any")
    .drop(columns=["FF_ROIC", "FF_ROE"])
)

# sector中立でROICをランキング
df_roic_merged = roic_utils.add_factor_rank_cols(df_roic_merged, factor_name="ROIC")
df_roic_merged = roic_utils.add_shifted_factor_cols_month(
    df_roic_merged,
    factor_name="ROIC_Rank",
    shift_month=list(range(1, 61)),
    shift_direction="Past",
)

# ROICラベルを付与
year_period = 5
df_roic_merged[f"ROIC_label_Past{year_period}Y"] = df_roic_merged.apply(
    lambda row: roic_utils.test_assign_roic_label(
        row=row,
        freq="annual",
        shift_direction="Past",
        year_period=year_period,
        judge_by_slope=False,
    ),
    axis=1,
)

df = (
    df_roic_merged.copy()[["date", "Symbol", "ROIC_label_Past5Y"]]
    .rename(columns={"ROIC_label_Past5Y": "value", "Symbol": "P_SYMBOL"})
    .dropna(subset=["value"], ignore_index=True)
    .assign(variable="ROIC_label_Past5Y", date=lambda row: pd.to_datetime(row["date"]))
)
display(df)


# データベース保存
f_db_utils.store_to_database(df=df, db_path=db_path, table_name="ROIC_label_Past5Y")

,date,P_SYMBOL,value,variable
0,2011-07-31,MRK-US,move to high,ROIC_label_Past5Y
1,2011-08-31,MRK-US,move to high,ROIC_label_Past5Y
2,2011-09-30,MRK-US,move to high,ROIC_label_Past5Y
3,2011-10-31,MRK-US,move to high,ROIC_label_Past5Y
4,2011-11-30,MRK-US,move to high,ROIC_label_Past5Y
...,...,...,...,...
83120,2025-09-30,WOW-AU,others,ROIC_label_Past5Y
83121,2025-09-30,WPP-GB,others,ROIC_label_Past5Y
83122,2025-09-30,WTB-GB,others,ROIC_label_Past5Y
83123,2025-09-30,YAR-NO,drop to low,ROIC_label_Past5Y


テーブル 'ROIC_label_Past5Y' は存在しません。新しいテーブルとして、すべての 83125 行を追加します。
  -> データの書き込みが完了しました。


In [ ]:
# --- label count ---
roic_count = pd.pivot(
    pd.DataFrame(
        df.groupby(["date", "GICS Sector", "ROIC_label_Past5Y"])["Symbol"].count()
    ).reset_index(),
    index=["date", "GICS Sector"],
    columns="ROIC_label_Past5Y",
).reset_index()
display(roic_count.loc[roic_count["GICS Sector"] == "Information Technology"])

roic_count = pd.pivot(
    pd.DataFrame(
        df.groupby(["date", "GICS Sector", "ROIC_Rank"])["Symbol"].count()
    ).reset_index(),
    index=["date", "GICS Sector"],
    columns="ROIC_Rank",
).reset_index()
display(roic_count.loc[roic_count["GICS Sector"] == "Information Technology"])

# --- weight check ---
weight_total_count = (
    df.groupby(["date"])["Weight (%)"]
    .agg(["count", "sum"])
    .rename(columns={"count": "Num of Securities", "sum": "Total Weight (%)"})
).sort_index()

weight_sector_count = (
    df.groupby(["date", "GICS Sector"])["Weight (%)"]
    .agg(["count", "sum"])
    .rename(columns={"count": "Num of Securities", "sum": "Total Weight (%)"})
).sort_index()
display(weight_total_count)
display(weight_sector_count)

roic_label_count = (
    df.groupby(["date", "ROIC_label_Past5Y"])["Weight (%)"]
    .agg(["count", "sum"])
    .rename(columns={"count": "Num of Securities", "sum": "Total Weight (%)"})
).sort_index()
display(roic_label_count)

date             GICS Sector      Symbol               \
ROIC_label_Past5Y                                    drop to low move to high   
20                2013-08-31  Information Technology         3.0         10.0   
30                2013-09-30  Information Technology         3.0         10.0   
40                2013-10-31  Information Technology         2.0         10.0   
50                2013-11-30  Information Technology         3.0         11.0   
60                2013-12-31  Information Technology         5.0         10.0   
...                      ...                     ...         ...          ...   
1018              2025-05-31  Information Technology        11.0          9.0   
1029              2025-06-30  Information Technology        10.0          7.0   
1040              2025-07-31  Information Technology        11.0          7.0   
1051              2025-08-31  Information Technology        13.0          7.0   
1062              2025-09-30  Information Technology         5.0          1.0   

                                                 
ROIC_label_Past5Y others remain high remain low  
20                  24.0        10.0        3.0  
30                  24.0        11.0        4.0  
40                  25.0        10.0        5.0  
50                  24.0        10.0        5.0  
60                  21.0        11.0        6.0  
...                  ...         ...        ...  
1018                32.0        13.0        2.0  
1029                33.0        15.0        2.0  
1040                31.0        16.0        1.0  
1051                30.0        16.0        1.0  
1062                 9.0         7.0        1.0  

[98 rows x 7 columns]

date             GICS Sector Symbol                        
ROIC_Rank                                     rank1 rank2 rank3 rank4 rank5
7         2005-08-31  Information Technology     25    25    24    25    25
17        2005-09-30  Information Technology     25    25    25    25    25
27        2005-10-31  Information Technology     25    25    25    25    26
37        2005-11-30  Information Technology     25    25    25    25    26
47        2005-12-31  Information Technology     25    25    25    25    25
...              ...                     ...    ...   ...   ...   ...   ...
1605      2025-05-31  Information Technology     24    23    24    23    24
1616      2025-06-30  Information Technology     23    22    23    22    23
1627      2025-07-31  Information Technology     23    22    22    22    23
1638      2025-08-31  Information Technology     23    22    22    22    23
1648      2025-09-30  Information Technology      8     7     7     7     8

[158 rows x 7 columns]

,Num of Securities,Total Weight (%)
date,,
2005-08-31,1296,91.341262
2005-09-30,1291,91.723118
2005-10-31,1290,91.461135
2005-11-30,1284,91.742185
2005-12-31,1318,95.426320
...,...,...
2025-05-31,1160,99.764161
2025-06-30,1129,99.163967
2025-07-31,1127,99.286490


Num of Securities  Total Weight (%)
date       GICS Sector                                                
2005-08-31 Communication Services                 40          4.554126
           Consumer Discretionary                231         10.119443
           Consumer Staples                       91          8.564437
           Energy                                 73         10.562697
           Financials                            280         21.092156
...                                              ...               ...
2025-09-30 Industrials                            68          2.657545
           Information Technology                 37         12.303719
           Materials                              25          1.015295
           Real Estate                            20          0.264640
           Utilities                              18          0.251028

[1653 rows x 2 columns]

Num of Securities  Total Weight (%)
date       ROIC_label_Past5Y                                     
2011-07-31 move to high                       1          0.466623
2011-08-31 move to high                       1          0.487915
2011-09-30 move to high                       1          0.533305
2011-10-31 move to high                       1          0.505093
2011-11-30 move to high                       1          0.537068
...                                         ...               ...
2025-09-30 drop to low                       35          3.672414
           move to high                      64          2.705728
           others                           117          9.559750
           remain high                       54         11.495302
           remain low                        29          1.265848

[503 rows x 2 columns]

In [ ]:
# --- add ROIC Rank cols ---
df_roic_merged = roic_utils.add_shifted_roic_cols_month(
    df_roic_merged, shift_direction="Past"
)
df_roic_merged = roic_utils.add_roic_rank_cols(df_roic_merged, freq_suffix="1M")
df_roic_merged = roic_utils.add_roic_rank_cols(df_roic_merged, freq_suffix="12M")
df_roic_merged = roic_utils.add_roic_rank_cols(df_roic_merged, freq_suffix="24M")
df_roic_merged = roic_utils.add_roic_rank_cols(df_roic_merged, freq_suffix="36M")
df_roic_merged = roic_utils.add_roic_rank_cols(df_roic_merged, freq_suffix="48M")
df_roic_merged = roic_utils.add_roic_rank_cols(df_roic_merged, freq_suffix="60M")
df_roic_merged["ROIC_label_Past5Y"] = df_roic_merged.apply(
    lambda row: roic_utils.test_assign_roic_label(
        row=row,
        freq="annual",
        shift_direction="Past",
        year_period=5,
        judge_by_slope=False,
    ),
    axis=1,
)
df = df_roic_merged.copy()[
    [
        "date",
        "Symbol",
        "FG_COMPANY_NAME",
        "GICS Sector",
        "GICS Industry",
        "Weight (%)",
        "Mkt Value",
        "ROIC_label_Past5Y",
    ]
]

# --- 作成したROIC特徴量をデータベースに保存 ---
df_to_store = (
    df[["date", "Symbol", "ROIC_label_Past5Y"]]
    .rename(columns={"Symbol": "P_SYMBOL", "ROIC_label_Past5Y": "value"})
    .assign(variable="ROIC_label_Past5Y")
)
f_db_utils.store_to_database(
    df=df_to_store,
    db_path=db_path,
    table_name="ROIC_label_Past5Y",
    unique_cols=["date", "P_SYMBOL"],
)
del df_roic_merged, df_to_store


# --- label count ---
roic_count = pd.pivot(
    pd.DataFrame(
        df.groupby(["date", "GICS Sector", "ROIC_label_Past5Y"])["Symbol"].count()
    ).reset_index(),
    index=["date", "GICS Sector"],
    columns="ROIC_label_Past5Y",
)
display(roic_count)
weight_total_count = (
    df.groupby(["date"])["Weight (%)"]
    .agg(["count", "sum"])
    .rename(columns={"count": "Num of Securities", "sum": "Total Weight (%)"})
).sort_index()
weight_sector_count = (
    df.groupby(["date", "GICS Sector"])["Weight (%)"]
    .agg(["count", "sum"])
    .rename(columns={"count": "Num of Securities", "sum": "Total Weight (%)"})
).sort_index()
display(weight_total_count)
display(weight_sector_count)

roic_label_count = (
    df.groupby(["date", "ROIC_label_Past5Y"])["Weight (%)"]
    .agg(["count", "sum"])
    .rename(columns={"count": "Num of Securities", "sum": "Total Weight (%)"})
).sort_index()
display(roic_label_count)

既存の 144706 行との重複をチェックしました。55963 行を新たに追加します。
  -> データの書き込みが完了しました。


Symbol                                  \
ROIC_label_Past5Y                 drop to low move to high others remain high   
date       GICS Sector                                                          
2013-08-31 Communication Services         2.0          NaN    7.0         NaN   
           Consumer Discretionary        28.0          8.0   20.0        13.0   
           Consumer Staples               5.0          6.0   19.0        10.0   
           Energy                         3.0          9.0   14.0         4.0   
           Financials                    18.0         29.0   56.0        16.0   
...                                       ...          ...    ...         ...   
2025-09-30 Industrials                    7.0          8.0   17.0        11.0   
           Information Technology         6.0          NaN    9.0         7.0   
           Materials                      4.0          4.0    9.0         2.0   
           Real Estate                    NaN          3.0    7.0         NaN   
           Utilities                      1.0          NaN    9.0         NaN   

                                              
ROIC_label_Past5Y                 remain low  
date       GICS Sector                        
2013-08-31 Communication Services        3.0  
           Consumer Discretionary        8.0  
           Consumer Staples              5.0  
           Energy                        6.0  
           Financials                    6.0  
...                                      ...  
2025-09-30 Industrials                  10.0  
           Information Technology        1.0  
           Materials                     1.0  
           Real Estate                   7.0  
           Utilities                     6.0  

[1053 rows x 5 columns]

,Num of Securities,Total Weight (%)
date,,
2005-08-31,1296,91.341262
2005-09-30,1291,91.723118
2005-10-31,1290,91.461135
2005-11-30,1284,91.742185
2005-12-31,1318,95.426320
...,...,...
2025-05-31,1160,99.764161
2025-06-30,1129,99.163967
2025-07-31,1127,99.286490


Num of Securities  Total Weight (%)
date       GICS Sector                                                
2005-08-31 Communication Services                 40          4.554126
           Consumer Discretionary                231         10.119443
           Consumer Staples                       91          8.564437
           Energy                                 73         10.562697
           Financials                            280         21.092156
...                                              ...               ...
2025-09-30 Industrials                            68          2.657545
           Information Technology                 37         12.303719
           Materials                              25          1.015295
           Real Estate                            20          0.264640
           Utilities                              18          0.251028

[1653 rows x 2 columns]

Num of Securities  Total Weight (%)
date       ROIC_label_Past5Y                                     
2013-08-31 drop to low                       99          9.008378
           move to high                      95         10.276710
           others                           208         20.954615
           remain high                       78         15.244679
           remain low                        76          5.360637
...                                         ...               ...
2025-09-30 drop to low                       41          4.949732
           move to high                      45          1.852377
           others                           130          9.670608
           remain high                       49         11.410937
           remain low                        34          0.815388

[490 rows x 2 columns]

## Merge(ROIC label + Return data)


In [ ]:
db_path = DATA_DIR / "MSCI_KOKUSAI_Return.db"
conn = sqlite3.connect(db_path)
df_return = pd.read_sql(
    "SELECT * FROM Annualized_Return", con=conn, parse_dates=["date"]
)
df_return = pd.pivot(
    df_return, index=["date", "Symbol"], columns="variable", values="value"
)
conn.close()

db_path = DATA_DIR / "MSCI_KOKUSAI_Custom_Feature.db"
conn = sqlite3.connect(db_path)
df_roic = pd.read_sql("SELECT * FROM ROIC_label_Past5Y", con=conn, parse_dates=["date"])
conn.close()

# display(df_return)
# display(df_roic)
print(f"Return dataframe shape: {df_return.shape}")
print(f"ROIC label dataframe shape: {df_roic.shape}")


df = (
    pd.merge(df_roic, df_return, on=["date", "Symbol"], how="left")
    .sort_values(["date", "Symbol"], ignore_index=True)
    .dropna(subset=["Weight (%)", "ROIC_label_Past5Y"], how="any")
)
del df_roic, df_return
print(f"Merged dataframe shape: {df.shape}")
display(df)

Return dataframe shape: (676607, 6)
ROIC label dataframe shape: (217818, 8)
Merged dataframe shape: (96159, 14)


,date,Symbol,FG_COMPANY_NAME,GICS Sector,GICS Industry,Weight (%),Mkt Value,ROIC_label_Past5Y,Return_Ann_1M,Return_Ann_1Y,Return_Ann_3M,Return_Ann_3Y,Return_Ann_5Y,Return_Ann_6M
78601,2012-08-31,0P7J-GB,Amec Foster Wheeler plc,Energy,Energy Equipment & Services,0.026104,4.570660e+11,drop to low,0.066922,0.169121,0.678064,0.256792,0.193707,-0.006009
78604,2012-08-31,1038-HK,CKインフラストラクチャー・ホールディングス,Utilities,Electric Utilities,0.012952,2.267820e+11,drop to low,-0.067819,-0.002079,0.382333,0.148123,0.102742,0.056078
78605,2012-08-31,11-HK,ハン・セン・バンク,Financials,Banks,0.048676,8.522970e+11,remain high,0.270841,-0.033913,0.386150,0.080675,0.004829,0.032736
78607,2012-08-31,12-HK,ヘンダーソン・ランド・デベロップメント,Financials,Real Estate Management & Development,0.026059,4.562850e+11,move to high,0.641988,0.052615,0.797608,0.088944,0.012567,-0.041637
78610,2012-08-31,13.XX1-HK,Hutchison Whampoa Limited,Industrials,Industrial Conglomerates,0.083389,1.460100e+12,remain low,-0.361218,-0.091214,0.245423,0.129290,-0.025497,-0.255749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217812,2025-07-31,Z74-SG,シンガポール・テレコミュニケーションズ,Communication Services,Diversified Telecommunication Services,0.030780,3.344860e+12,others,-0.034226,0.538649,0.130107,0.184226,0.041258,0.562300
217813,2025-07-31,ZAL-DE,ザランド,Consumer Discretionary,Specialty Retail,0.009153,9.946100e+11,others,-1.301110,0.111731,-0.842763,-0.326717,-0.059330,-0.487816
217814,2025-07-31,ZBH-US,ジンマー・バイオメット・ホールディングス,Health Care,Health Care Equipment & Supplies,0.025111,2.728800e+12,remain low,0.057749,-0.194701,-0.468949,-0.099124,-0.074728,-0.355530
217816,2025-07-31,ZTS-US,ゾエティス Class A,Health Care,Pharmaceuticals,0.090062,9.786870e+12,remain high,-0.808419,-0.211012,-0.280998,-0.094628,0.031428,-0.317823


### Check constituents


In [ ]:
g = df.groupby(["date"])["Weight (%)"].agg(["count", "sum"])
display(g.tail(10))
display(
    df.loc[
        (df["FG_COMPANY_NAME"].str.contains("NVIDIA"))
        & (df["GICS Sector"] == "Information Technology")
    ]
)

,count,sum
date,,
2024-07-31,918,92.119993
2024-09-30,909,91.925236
2024-10-31,909,91.803620
2024-12-31,902,91.681202
2025-01-31,904,91.444891
2025-02-28,918,92.131118
2025-03-31,897,92.223468
2025-04-30,901,91.909803
2025-06-30,797,88.918013


,date,Symbol,FG_COMPANY_NAME,GICS Sector,GICS Industry,Weight (%),Mkt Value,ROIC_label_Past5Y,Return_Ann_1M,Return_Ann_1Y,Return_Ann_3M,Return_Ann_3Y,Return_Ann_5Y,Return_Ann_6M
79401,2012-08-31,NVDA-US,NVIDIAコーポレーション,Information Technology,Semiconductors & Semiconductor Equipment,0.038312,6.708320e+11,others,0.426596,0.052682,0.484340,0.117571,-0.112797,-0.153605
80629,2012-10-31,NVDA-US,NVIDIAコーポレーション,Information Technology,Semiconductors & Semiconductor Equipment,0.032390,5.920480e+11,others,-1.295350,-0.211806,-0.491308,0.019635,-0.109140,-0.164256
81855,2012-11-30,NVDA-US,NVIDIAコーポレーション,Information Technology,Semiconductors & Semiconductor Equipment,0.032106,6.113730e+11,others,-0.005013,-0.266789,-0.635177,-0.025716,-0.094363,-0.075419
83100,2012-12-31,NVDA-US,NVIDIAコーポレーション,Information Technology,Semiconductors & Semiconductor Equipment,0.032503,6.582890e+11,others,0.287262,-0.122665,-0.337700,-0.056395,-0.116272,-0.239550
84361,2013-01-31,NVDA-US,NVIDIAコーポレーション,Information Technology,Semiconductors & Semiconductor Equipment,0.030846,6.930990e+11,others,0.000000,-0.186256,0.094083,-0.067902,-0.126635,-0.198613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211945,2025-01-31,NVDA-US,NVIDIAコーポレーション,Information Technology,Semiconductors & Semiconductor Equipment,4.301899,4.560820e+14,remain high,-1.343121,1.079883,-0.401872,0.605904,0.701776,0.051460
213137,2025-02-28,NVDA-US,NVIDIAコーポレーション,Information Technology,Semiconductors & Semiconductor Equipment,4.510090,4.617730e+14,remain high,0.475184,0.982432,-0.405560,0.573094,0.695583,0.090891
214305,2025-03-31,NVDA-US,NVIDIAコーポレーション,Information Technology,Semiconductors & Semiconductor Equipment,4.121013,3.969130e+14,remain high,-1.704360,0.566167,-0.857432,0.551579,0.635205,-0.227553
215464,2025-04-30,NVDA-US,NVIDIAコーポレーション,Information Technology,Semiconductors & Semiconductor Equipment,4.121320,3.804850e+14,remain high,0.059641,0.319749,-0.389845,0.401324,0.694163,-0.395859


### ROIC label Performance


In [ ]:
avg_return = (
    pd.DataFrame(
        df.groupby(["date", "ROIC_label_Past5Y"]).apply(
            lambda row: roic_utils.clipped_mean(row["Return_Ann_5Y"], percentile=5)
        )
    )
    .reset_index()
    .rename(columns={0: "avg"})
)
std = pd.DataFrame(
    df.groupby(["date", "ROIC_label_Past5Y"])
    .apply(lambda row: roic_utils.clipped_std(row["Return_Ann_5Y"], percentile=5))
    .reset_index()
    .rename(columns={0: "std"})
)
df_performance = pd.merge(
    avg_return, std, on=["date", "ROIC_label_Past5Y"], how="left"
).assign(efficiency=lambda row: row["avg"].div(row["std"]))
df_performance = pd.pivot(
    df_performance, index="date", columns="ROIC_label_Past5Y"
).sort_index()
display(df_performance)

avg                                                \
ROIC_label_Past5Y drop to low move to high    others remain high remain low   
date                                                                          
2012-08-31           0.038617    -0.078085 -0.013442    0.064215  -0.047445   
2012-10-31           0.033293    -0.072137 -0.014087    0.064566  -0.050651   
2012-11-30           0.031163    -0.075659 -0.017843    0.058905  -0.052669   
2012-12-31           0.025686    -0.074313 -0.023291    0.044640  -0.049142   
2013-01-31           0.023791    -0.073737 -0.007586    0.061751  -0.048795   
...                       ...          ...       ...         ...        ...   
2025-02-28           0.130007     0.023283  0.061586    0.137381   0.019253   
2025-03-31           0.130334     0.023350  0.062636    0.125173   0.016550   
2025-04-30           0.137255     0.021089  0.067438    0.126168   0.022981   
2025-06-30           0.157507     0.035203  0.090054    0.124221   0.023517   
2025-07-31           0.152448     0.058672  0.080205    0.129478   0.032082   

                          std                                                \
ROIC_label_Past5Y drop to low move to high    others remain high remain low   
date                                                                          
2012-08-31           0.079274     0.127608  0.099918    0.081558   0.108129   
2012-10-31           0.071016     0.115481  0.096288    0.076514   0.104184   
2012-11-30           0.064521     0.118413  0.100145    0.079537   0.100199   
2012-12-31           0.062965     0.113466  0.095717    0.070713   0.093004   
2013-01-31           0.059179     0.119810  0.094802    0.072411   0.097013   
...                       ...          ...       ...         ...        ...   
2025-02-28           0.091646     0.098077  0.096638    0.083625   0.079172   
2025-03-31           0.081238     0.094644  0.093260    0.080353   0.077009   
2025-04-30           0.094701     0.092541  0.095933    0.081710   0.079056   
2025-06-30           0.099677     0.097340  0.090821    0.087251   0.084390   
2025-07-31           0.098593     0.103705  0.095187    0.078827   0.084529   

                   efficiency                                                
ROIC_label_Past5Y drop to low move to high    others remain high remain low  
date                                                                         
2012-08-31           0.487136    -0.611917 -0.134533    0.787346  -0.438784  
2012-10-31           0.468812    -0.624667 -0.146297    0.843846  -0.486166  
2012-11-30           0.482988    -0.638942 -0.178175    0.740598  -0.525648  
2012-12-31           0.407938    -0.654933 -0.243331    0.631284  -0.528388  
2013-01-31           0.402018    -0.615449 -0.080019    0.852787  -0.502971  
...                       ...          ...       ...         ...        ...  
2025-02-28           1.418581     0.237397  0.637283    1.642818   0.243183  
2025-03-31           1.604357     0.246717  0.671634    1.557788   0.214911  
2025-04-30           1.449345     0.227893  0.702965    1.544097   0.290696  
2025-06-30           1.580175     0.361653  0.991552    1.423716   0.278674  
2025-07-31           1.546232     0.565763  0.842598    1.642561   0.379536  

[111 rows x 15 columns]